In [1]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input
from keras.utils import np_utils
from keras.layers.core import Dense
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
#data = pd.read_csv('kddcup.data_10_percent_corrected')
#data
labels = pd.read_csv("labels.csv",sep = ":",header = None)
data   = pd.read_csv("kddcup.data_10_percent_corrected", names = labels.iloc[:,0].values)
data_onehotencoded = data.copy()

In [ ]:
lb_make = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
cat_columns = labels.loc[labels.iloc[:,1] == " symbolic.",0].values
data[cat_columns] = data[cat_columns].apply(lambda col: lb_make.fit_transform(col))

In [6]:
data.iloc[:,-1] = np.where(data.iloc[:,-1] != 'normal.',1,data.iloc[:,-1])
data.iloc[:,-1] = np.where(data.iloc[:,-1] == 'normal.',0,data.iloc[:,-1])

data.iloc[:,-1].unique()

array([0, 1], dtype=object)

In [ ]:
def ffr(df, t):
    classes = ['normal', 'u2r', 'dos', 'r2l', 'probe']
    classes2 = [0,1]
    df_new = df.iloc[:,:-1]
    var_d = []
    for clas in classes2:
        temp = df.loc[df['output'] == clas]
        #print("is unique ", temp.output.unique())
        temp = temp.iloc[:,:-1].values
        #print("temp = ", temp)
        min_max_scaler = preprocessing.MinMaxScaler()
        temp_scaled = min_max_scaler.fit_transform(temp)
        #print("temp_scaled = ", temp_scaled)
        mean_ind = temp_scaled.mean(axis = 0)
        #print("mean_ind = ", mean_ind)
        var_d_f = np.square(mean_ind - temp).mean(axis = 0)
        #print("var_d_f_f = ", var_d_f)
        var_d.append(var_d_f)
    var_d = np.array(var_d)
    #print(var_d)
    var_d_mean = var_d.mean(axis = 0)
    
    
#     var = np.zeros(len(mean_means))
#     for i in means:
#         var += np.square(i - mean_means)
#     var /= len(mean_means)
#     #print(var)
    indexes = list(np.argsort(var_d_mean))
    filtered_indexes = indexes[:t]
    filtered_indexes.append(41)
    filtered_data = df.iloc[:, filtered_indexes]
    return filtered_data

In [ ]:
def NN(data,feature):
    epoch = 100
    batchsize = 32

    data = data.sample(frac=1,random_state=200)
    Input1 = data.iloc[:,:41]
    Output1 = data.iloc[:,-1]
    Input1 = np.array(Input1)
    Output1 = np.array(Output1)
    Output1 = Output1.reshape(Output1.shape[0],1)
    print(np.unique(Output1))
    print(Input1.shape)
    print(Output1.shape)
    print(type(Input1))
    print(type(Output1))
    Input1 = Input1.astype(float)
    split = int(math.ceil(0.8 * len(Input1)))
    X_train = Input1[0:split]
    Y_train = Output1[0:split]

    X_test = Input1[split:]
    Y_test = Output1[split:]

    print(X_train.shape)
    print(X_test.shape)
    print(Y_train.shape)
    print(Y_test.shape)
    flatten_Y_train = Y_train.ravel()
    class_weights = class_weight.compute_class_weight('balanced' ,np.unique(Y_train) ,flatten_Y_train)
    inp = Input(shape=(feature,))
    hidden1 = Dense(40,activation='sigmoid')(inp)
    hidden2 = Dense(40,activation='sigmoid')(hidden1)
    out = Dense(1,activation='sigmoid')(hidden2)

    model = Model(inp,out)
    model.compile(optimizer='adam',loss='binary_crossentropy')
    model.summary()
    import time

    s = time.time()
    model.fit(X_train,Y_train,epochs = epoch,batch_size=batchsize,class_weight=class_weights)
    e = time.time()
    print("Training Time for Binary NN : ", (e - s))
    
    predicted = model.predict(X_test)
    predicted = predicted.ravel()
    #print(len(predicted))
    Y_predicted = np.empty((len(predicted),))
    for i in range(len(predicted)):
        if predicted[i] < 0.5:
            Y_predicted[i] = 0
            #Y_predicted[i] = (Y_predicted[i])
        else:
            Y_predicted[i] = 1
            #Y_predicted[i] = math.floor(Y_predicted[i])

    Y_predicted = Y_predicted.astype(int)
    Y_test_final = np.array(Y_test)
    print(Y_predicted.shape)
    print(Y_test_final.shape)
    print(type(Y_predicted))
    print(type(Y_test_final))

    print("Y_predicted Unique and its Frequency :")
    y_pre,y_pred_counts = np.unique(Y_predicted,return_counts=True)
    print(y_pre,y_pred_counts)

    print("Y_test Final unique and its Frequency : ")
    y_test_final,y_test_counts = np.unique(Y_test_final,return_counts=True)
    print(y_test_final,y_test_counts)


    tp,fp,tn,fn = 0,0,0,0
    for i in range(len(Y_test_final)):
        if Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 1:
            tp = tp + 1
        elif Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 0:
            tn = tn + 1
        elif Y_test_final[i] != Y_predicted[i] and Y_test_final[i] == 0:
            fp = fp + 1
        else:
            fn = fn + 1

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = (2*precision*recall)/(precision + recall)
    print("tp :",tp)
    print("tn :",tn)
    print("fp :",fp)
    print("fn :",fn)
    print("Accuracy : ",accuracy)
    print("Precision : ",precision)
    print("Recall : ",recall)
    print("f1_score : ",f1_score)

    

In [ ]:
from sklearn import preprocessing
for i in range(10, 31, 10):
    new_dat = ffr(data, i)
    print("features = ", i)
    print(new_dat.shape)
    print(type(new_dat))
    print("With Feature size : ", i)
    NN(new_dat,i)

In [12]:
# np.savetxt("output_file",Y_predicted)

In [13]:
# predicted = predicted.ravel()
# Y_test = Y_test.ravel()
# print(predicted.shape)
# print(Y_test.shape)
# Y_test_final = np.empty((len(Y_test),))
# for i in range(len(Y_test)):
#     Y_test_final[i] = np.argmax(Y_test[i])

# Y_predicted = np.empty((len(predicted),))
# for i in range(len(predicted)):
#     if predicted[i] < 0.5:
#         Y_predicted[i] = int(0)
#     else:
#         Y_predicted[i] = int(1)

# #print(Y_test_final.shape)
# print(Y_predicted.shape)

# print("Predicted Unique and their Freqencies : ")
# pred,pred_count = np.unique(predicted,return_counts=True)
# print(pred,pred_count)
# print("Y_predicted Unique and its Frequency :")
# y_pre,y_pred_counts = np.unique(Y_predicted,return_counts=True)
# print(y_pre,y_pred_counts)
# print("Y_test unique and its Frequency : ")
# y_test,y_test_counts = np.unique(Y_test,return_counts=True)
# print(y_test,y_test_counts)
# print("Y_test Final unique and its Frequency : ")
# y_test_final,y_test_counts = np.unique(Y_test,return_counts=True)
# print(y_test_final,y_test_counts)
# #conf = confusion_matrix(Y_test,predicted)
# conf = confusion_matrix(Y_test_final,Y_predicted)
# print(conf)
# precision,recall,f1,support = precision_recall_fscore_support(Y_test_final,Y_predicted)
# print("Precision : ",precision)
# print("Recall : ",recall)
# print("f1 : ",f1)
# print("Support : ",support)